#### 数据记录
* 数据点总数：144
* Poi个数：18个，非poi个数：126个。
* feature数量：20个（'poi'变量作为label，不算做feature）
* feature中有NaN值的:
    * 共有50个salary为nan
    * 共有21个total_payments为nan
    * 共有79个long_term_incentive为nan
    * 21个total_stock_value为nan
    * LOCKHART EUGENE E对应的所有features都是NaN


##### 异常点:
* 打印所有的name，发现两个不属于人名的点：THE TRAVEL AGENCY IN THE PARK, TOTAL
* 分析salary和bonus两个变量的散点图，发现两个点对应的salary和bonus都远高于其他点，打印姓名为SKILLING JEFFREY K和LAY KENNETH L。打印两个人的poi属性，发现都=1.将这两个点作为异常值，使用dict.pop()从数据集中删除
* salary-bonus散点图如下:

<img src="./Figure.png" width=400 height=300>

#### 优化和选择特征

##### 增加新的变量 ratio_email_with_poi
* 根据邮件中与POI交互邮件所占比来分析是否是poi.比值越大，是poi的嫌疑越大.
* 计算ratio_email_with_poi并绘制与变量poi的箱线图.
* 增加ratio_from_this_to_poi, ratio_from_poi_to_this, 代表用户与poi之间的邮件往来频率.相比ratio_email_with_poi更精确一些

<img src="./Figure_1.png" width=400 height=300>

##### 特征选择


<img src="./features.png" width=400 height=400>

* 如何选择features：
    * 对数据集进行决策树训练，从训练模型的变量权重中，提取出权重比较高的变量作为新的变量集合再次训练。如果最终准确率精确率都得到明显提升，则采用这个变量集合。
* 对原数据集合和全部变量采用SelectKBest选取最优变量，继续进行决策树模型训练，准确率和精确率没有明显上升。


* 操作过程：
    * 第一次使用全部特征，通过DecisionTreeClassifier.feature_importance_得到比较重要的特征，再迭代进决策树模型。
    * 通过几次迭代发现特征并不太理想，随后使用增加后的变量，再不断的调试模型，最终决定采用['ratio_to_poi','ratio_from_poi','total_payments','total_stock_value','long_term_incentive']
    * 使用新的数据集和这个变量集合，本地测试决策树模型，结果：准确率0.893617 精度0.500000 召回率0.600000 f1 0.545455
    * 测试tester.py，结果 Accuracy: 0.82073	Precision: 0.32715	Recall: 0.32600	F1: 0.32657	F2: 0.32623


##### 特征缩放：
* 对SVC模型进行特征缩放。
* 对选取的feature列表，使用min_max缩放器对数值进行缩放，使所有的feature都在0-1之间
    * 'deferral_payments', 'total_payments', 'loan_advances', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value'
    * 上面这几个变量变化区间过大，因此对他们进行缩放

#### 选择算法和调整参数

##### 调整参数
* 就是对模型中的一些参数进行动态调整，以使模型达到最优。在本项目的解答中，通过设定参数random_state对决策树模型进行调整。

##### 选择其他算法：
* 朴素贝叶斯：准确率0.72,精确率0.2，召回率0.2
* 支持向量机: 准确率0.69, 精确率0, 召回率0

* 最终选择决策树模型，因为准确率和精确率较高。

#### 验证和评估
* 验证是从原始数据中分离出一部分作为测试数据，来评估模型的准确率。本项目解答中，在训练时通过sklearn包中的相关模块，将原始数据分割为训练数据和测试数据，再通过accuracy_score模块对最终的训练模型精确率进行评估; 二次训练时使用GridSearchCV对模型进行交叉验证。
* 未进行验证，结果会过拟合。所有数据都是训练数据，模型会尽量偏重每个数据的正确性，从而导致模型过拟合。

* 准确率：模型预测正确的/全部。
* 假设：预测结果是真--PT，预测结果是假--PF,实际是真--TT, 实际是假--TF
   * 精确率：TT&&PT/(TT&&PT+TF&&PT)
   * 召回率: TT&&PT/(TT&&PT+TT&&PF)
   

* precision=0.2表示是poi并且被预测为poi的人占所有被预测为poi的人的20%
* recall=0.16表示是poi并且被预测为poi的人占是poi的人的预测结果的16%

* tester.py中的startifiedshufflesplit:
* sklearn.cross_validation.startifiedshufflesplit:
    * shufflesplit: 一个迭代器，生成指定数量的训练集和测试集。可以设置迭代次数和测试集百分比。
    * startified: 使用同样的迭代次数\测试集百分比，来对两个数据集(features和targets)进行分割。